In [6]:
from binance.client import Client as bnb_client
import pandas as pd

client = bnb_client(tld='US')
def get_crypto_data(symbol):
    # Kline (the information we are working with): open price, high price, low price, close price, volume in the four hour period
    klines = client.get_historical_klines(symbol, '4h', '2020-01-01', '2024-12-31')
    
    df = pd.DataFrame(klines)
    df[0] = pd.to_datetime(df[0], unit='ms')
    df = df.set_index(0)[[4]]  # Just close price
    df.columns = [symbol]
    df[symbol] = df[symbol].astype(float)
    
    return df

btc_data = get_crypto_data('BTCUSDT')
print(btc_data.head())
print(f"Shape: {btc_data.shape}")

                     BTCUSDT
0                           
2020-01-01 00:00:00  7230.71
2020-01-01 04:00:00  7205.50
2020-01-01 08:00:00  7195.80
2020-01-01 12:00:00  7233.02
2020-01-01 16:00:00  7223.72
Shape: (10951, 1)


In [10]:
symbols = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'BNBUSDT', 'XRPUSDT', 'DOTUSDT', 'MATICUSDT', 'LINKUSDT']
dfs = []


for symbol in symbols:
    df = get_crypto_data(symbol)
    dfs.append(df)


# combining into one data frame

prices = pd.concat(dfs, axis=1)
print(prices.head())

                     BTCUSDT  ETHUSDT  ADAUSDT  BNBUSDT  XRPUSDT  DOTUSDT  \
0                                                                           
2020-01-01 00:00:00  7230.71   130.18  0.03308  13.8159  0.19406      NaN   
2020-01-01 04:00:00  7205.50   130.52  0.03320  13.7648  0.19518      NaN   
2020-01-01 08:00:00  7195.80   130.84  0.03321  13.7162  0.19358      NaN   
2020-01-01 12:00:00  7233.02   131.84  0.03357  13.7958  0.19428      NaN   
2020-01-01 16:00:00  7223.72   131.98  0.03361  13.7270  0.19474      NaN   

                     MATICUSDT  LINKUSDT  
0                                         
2020-01-01 00:00:00        NaN       NaN  
2020-01-01 04:00:00        NaN       NaN  
2020-01-01 08:00:00        NaN       NaN  
2020-01-01 12:00:00        NaN       NaN  
2020-01-01 16:00:00        NaN       NaN  


Because there is such a difference in price here, for our strategy we should use returns instead of just raw price data

In [11]:
returns = prices.pct_change().dropna()
print("Returns:")
print(returns.head())
print(f"Returns shape: {returns.shape}")
print("\nReturns stats:")
print(returns.describe())

Returns:
                      BTCUSDT   ETHUSDT   ADAUSDT   BNBUSDT  XRPUSDT  \
0                                                                      
2022-01-14 04:00:00 -0.001200 -0.001095  0.029176  0.007082      0.0   
2022-01-14 08:00:00 -0.013861 -0.019072 -0.038814 -0.024946      0.0   
2022-01-14 12:00:00  0.025944  0.022771  0.026040  0.019811      0.0   
2022-01-14 16:00:00 -0.000381  0.009870 -0.001944  0.009654      0.0   
2022-01-14 20:00:00 -0.000371 -0.000466  0.021628  0.008266      0.0   

                      DOTUSDT  MATICUSDT  LINKUSDT  
0                                                   
2022-01-14 04:00:00 -0.015693   0.006611 -0.010317  
2022-01-14 08:00:00 -0.021876  -0.031961 -0.034884  
2022-01-14 12:00:00  0.032600   0.039349  0.027005  
2022-01-14 16:00:00 -0.004772   0.013055  0.008495  
2022-01-14 20:00:00  0.024345   0.003436  0.034095  
Returns shape: (6491, 8)

Returns stats:
           BTCUSDT      ETHUSDT      ADAUSDT      BNBUSDT      XRPUSDT  \


/var/folders/rv/rzmfgtcs4dbc86tj60p89ly80000gn/T/ipykernel_28705/3953697242.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


In [16]:
crypto_columns = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'BNBUSDT', 'DOTUSDT', 'MATICUSDT', 'LINKUSDT']
clean_returns = returns[crypto_columns]

# Reversal signal (1 period = immediate reversal)
reversal_signal = clean_returns.rolling(1).mean().rank(1)
reversal_signal = reversal_signal.subtract(reversal_signal.mean(1), 0)
reversal_signal = reversal_signal.divide(reversal_signal.abs().sum(1), 0)

# Momentum signal (5 periods = 20-hour momentum)
momentum_signal = clean_returns.rolling(5).mean().rank(1)
momentum_signal = momentum_signal.subtract(momentum_signal.mean(1), 0)
momentum_signal = momentum_signal.divide(momentum_signal.abs().sum(1), 0)

print("\nReversal signal (should have values):")
print(reversal_signal.dropna().head(3))
print("\nMomentum signal (first non-NaN rows):")
print(momentum_signal.dropna().head(3))


Reversal signal (should have values):
                      BTCUSDT   ETHUSDT  ADAUSDT   BNBUSDT   DOTUSDT  \
0                                                                      
2022-01-14 04:00:00 -0.083333  0.000000     0.25  0.166667 -0.250000   
2022-01-14 08:00:00  0.250000  0.166667    -0.25  0.000000  0.083333   
2022-01-14 12:00:00 -0.083333 -0.166667     0.00 -0.250000  0.166667   

                     MATICUSDT  LINKUSDT  
0                                         
2022-01-14 04:00:00   0.083333 -0.166667  
2022-01-14 08:00:00  -0.083333 -0.166667  
2022-01-14 12:00:00   0.250000  0.083333  

Momentum signal (first non-NaN rows):
                      BTCUSDT   ETHUSDT   ADAUSDT   BNBUSDT   DOTUSDT  \
0                                                                       
2022-01-14 20:00:00 -0.250000 -0.166667  0.250000  0.000000 -0.083333   
2022-01-15 00:00:00 -0.083333  0.000000 -0.250000 -0.166667  0.083333   
2022-01-15 04:00:00 -0.250000  0.000000 -0.083333  0.0

In [15]:
hour = clean_returns.index.hour
day_of_week = clean_returns.index.dayofweek

us_institutional = (hour >= 13) & (hour <= 21) & (day_of_week <= 4)  # Weekday 1pm-9pm UTC
asian_retail = (hour >= 22) | (hour <= 6) | (day_of_week >= 5)        # Night hours OR weekends

print("Time regime split:")
print(f"US institutional periods: {us_institutional.sum()}")
print(f"Asian retail periods: {asian_retail.sum()}")
print(f"Other periods: {len(clean_returns) - us_institutional.sum() - asian_retail.sum()}")

# Test the hypothesis: switch between momentum and reversal
final_signal = momentum_signal.copy()
final_signal.loc[asian_retail] = -reversal_signal.loc[asian_retail]  # Use REVERSAL during retail hours

print("\nFinal signal (first few rows):")
print(final_signal.dropna().head(3))

Time regime split:
US institutional periods: 1544
Asian retail periods: 3404
Other periods: 1543

Final signal (first few rows):
                     BTCUSDT  ETHUSDT  ADAUSDT  BNBUSDT  DOTUSDT  MATICUSDT  \
0                                                                             
2022-01-14 04:00:00     0.10     0.05     -0.1    -0.05     0.20      -0.00   
2022-01-14 20:00:00    -0.20    -0.15      0.1    -0.05    -0.10       0.05   
2022-01-15 00:00:00     0.05    -0.00      0.2     0.10     0.15      -0.10   

                     LINKUSDT   hour  day_of_week  
0                                                  
2022-01-14 04:00:00      0.15 -0.175       -0.175  
2022-01-14 20:00:00      0.00  0.200        0.150  
2022-01-15 00:00:00     -0.15 -0.050       -0.200  
